In [24]:
# ! pip install iotbr==0.1.0
import numpy as np
from iotbr import tru as tru
from iotbr import tru_pb as tru_pb
from iotbr import io_system as ios

In [17]:
# 1) Encontrar matrizes de demanda final (mE) e demanda total (mU) a preços básicos
#usando o métrodo do Prof. Dr. Guilhoto
mU, mE = tru_pb.D_total_pb('2019', '68')

In [18]:
#2) Estimar matriz a preços básicos para anos e níveis diferente
mU, mE = tru_pb.D_total_pb('2010', '20')

In [19]:
# 3) Estimar matriz a preços básicos para anos, níveis e unidades diferentes
mU, mE = tru_pb.D_total_pb('2010','51','t-1')

In [22]:
# 4) estimar a matrizes do sistema IO
year = '2010'
level = '68'
unit = 't'
mU, mE = tru_pb.D_total_pb(year,level,unit)


#Estimar matriz (D)
mV = tru.read_var(year,level,'P_matrix',unit).values.T
#Total produzido por produto
#vQ = np.sum(mV, axis=0)
vQ = tru.read_var(year,level,'PT',unit).values
mQChapeu = np.diagflat(1/vQ)
mD = np.dot(mV, mQChapeu)

#Estimar matriz (B)
#Total produzido por setor
vVBP = np.sum(mV, axis=1)
vX = np.copy(vVBP)
mXChapeu = np.diagflat(1/vX)
mB=np.dot(mU,mXChapeu)

mA = np.dot(mD,mB).astype(float)
mY = np.dot(mD,mE).astype(float)
mZ = np.dot(mD,mU).astype(float)
mI = np.eye(int(level))

mLeontief = np.linalg.inv(mI - mA)

mLeontief

array([[1.03120550e+00, 5.90855217e-02, 1.31050540e-02, ...,
        1.71297779e-03, 7.67611874e-03, 0.00000000e+00],
       [4.10371406e-03, 1.04658749e+00, 7.69782326e-03, ...,
        4.70786312e-04, 3.55466549e-03, 0.00000000e+00],
       [5.54903277e-03, 1.30481823e-02, 1.08105735e+00, ...,
        5.42740862e-04, 1.18713282e-03, 0.00000000e+00],
       ...,
       [2.11664665e-04, 4.09153978e-04, 2.17234021e-04, ...,
        1.02038011e+00, 7.87288278e-03, 0.00000000e+00],
       [1.50403556e-03, 1.51744638e-03, 1.37009686e-03, ...,
        1.81082519e-03, 1.00377326e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [25]:
# 5) estimar as matrizes do sistema IO automaticamente
sys = ios.system('2010','68','t')
mU = sys.mD_int_pb
mE = sys.mD_final_pb
mZ = sys.mD_int_pb_qua
mY = sys.mD_final_pb_qua
mL = sys.mLeontief

In [27]:
print(mL.shape)
mL

(68, 68)


array([[1.03120550e+00, 5.90855217e-02, 1.31050540e-02, ...,
        1.71297779e-03, 7.67611874e-03, 0.00000000e+00],
       [4.10371406e-03, 1.04658749e+00, 7.69782326e-03, ...,
        4.70786312e-04, 3.55466549e-03, 0.00000000e+00],
       [5.54903277e-03, 1.30481823e-02, 1.08105735e+00, ...,
        5.42740862e-04, 1.18713282e-03, 0.00000000e+00],
       ...,
       [2.11664665e-04, 4.09153978e-04, 2.17234021e-04, ...,
        1.02038011e+00, 7.87288278e-03, 0.00000000e+00],
       [1.50403556e-03, 1.51744638e-03, 1.37009686e-03, ...,
        1.81082519e-03, 1.00377326e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [28]:
# 6) estimar matriz leontief usando (VA) e (mZ)
import numpy as np
df = tru.read_var('2010','68','VA_table','t')
#df.loc[:,df.columns.str.contains('Valor da produção')]
vVA_pro = df['Valor da produção'].values
vX = np.copy(vVA_pro)
mXChapeu = np.diagflat(1/vX)

mZ = sys.mD_int_pb_qua
mA=np.dot(mZ,mXChapeu).astype(float)
mI = np.eye(len(df))
mLeontief = np.linalg.inv(mI - mA)
mB = mLeontief
mB

array([[1.03120550e+00, 5.90855217e-02, 1.31050540e-02, ...,
        1.71297779e-03, 7.67611874e-03, 0.00000000e+00],
       [4.10371406e-03, 1.04658749e+00, 7.69782326e-03, ...,
        4.70786312e-04, 3.55466549e-03, 0.00000000e+00],
       [5.54903277e-03, 1.30481823e-02, 1.08105735e+00, ...,
        5.42740862e-04, 1.18713282e-03, 0.00000000e+00],
       ...,
       [2.11664665e-04, 4.09153978e-04, 2.17234021e-04, ...,
        1.02038011e+00, 7.87288278e-03, 0.00000000e+00],
       [1.50403556e-03, 1.51744638e-03, 1.37009686e-03, ...,
        1.81082519e-03, 1.00377326e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [30]:
# 7) Todas as matrizes do sistema IO no formato df
mZ = sys.mZ
mY = sys.mY
mL = sys.mL
mT = sys.mT_int
mM = sys.mM_int
#mV = tru.read_var('2019','68','P_matrix','t').T
#vX = np.sum(mV, axis=1)
#vX

mT

,I_imp,ICMS,IPI,OI_liq_Sub
"0191\nAgricultura, inclusive o apoio à agricultura e a pós-colheita",144.497787,2984.301976,49.207507,2184.419936
"0192\nPecuária, inclusive o apoio à pecuária",67.545387,2098.28322,63.218639,1002.874846
0280\nProdução florestal; pesca e aquicultura,7.485352,252.16383,4.669596,160.000588
0580\nExtração de carvão mineral e de minerais não-metálicos,45.190511,185.40208,34.379344,347.969841
"0680\nExtração de petróleo e gás, inclusive as atividades de apoio",179.135465,622.965601,73.097543,3048.128123
...,...,...,...,...
8691\nSaúde pública,98.650455,1676.6383,103.316677,1236.38142
8692\nSaúde privada,203.212164,3080.215012,250.087283,1568.114223
"9080\nAtividades artísticas, criativas e de espetáculos",7.668853,240.02476,21.481084,381.097896
9480\nOrganizações associativas e outros serviços pessoais,61.614727,2282.462314,154.802442,2324.373881


In [31]:
sys.mT_final

,X_bens_serv,C_g,C_ong,C_f,FBKF,DE
mI_imp,0.000000,31.459075,0.221910,5883.939958,3998.882893,0.0
mICMS,10712.680977,931.475627,0.000000,144769.172298,11832.336903,0.0
mIPI,2495.879808,0.212541,0.000000,17084.749138,5352.662577,0.0
mOI_liq_Sub,10778.643971,1131.691354,1236.539125,67666.853335,30095.420833,0.0


In [35]:
# 8) verificar se os calculos estão corretos
# Oferta agragada (AO) deve ser igual à demanda agragada (AD)
import numpy as np
from iotbr import io_system as ios
sys = ios.system('2011','51','t')
print('Oferta agragada (OA) estimada: ' + str(sys.OA))
print('Demanda agragada (DA) estimada: ' + str(sys.DA))

#Oferta agragada (OA) calculada pelo (IBGE)
OA = np.sum(tru.read_var('2011','51','PT','t'))
print('Oferta agragada (OA) calculada pelo (IBGE): '+ str(OA.values[0]))

#problema com o nível 51 (o problema está na tabela 'P_matrix'. Ela está deslocada uma coluna para frente. Corrigir o Dicionário)
#Oferta agragada (OA) estimada: 6599149.0
#Demanda agragada (DA) estimada: 6599149.0
#Oferta agragada (OA) calculada pelo (IBGE): 6599149

#Oferta agragada (OA) estimada: 4285219.0
#Demanda agragada (DA) estimada: 6545795.609601207
#Oferta agragada (OA) calculada pelo (IBGE): 6599149
#PT   -2313930.0

sys.OA - OA

Oferta agragada (OA) estimada: 7438007.0
Demanda agragada (DA) estimada: 7438007.0
Oferta agragada (OA) calculada pelo (IBGE): 7438007


/home/felipe/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


PT    0.0
dtype: object

In [36]:
# 9) Verificar igualdade entre demanda e oferta (niveis 12 e 51)
for t in range(2000,2011):
  print(t)
  [print(ios.system(str(t),x,'t').OA - ios.system(str(t),x,'t').DA) for x in ['12','51']]

2000
-2.3283064365386963e-10
-2.3283064365386963e-10
2001
4.656612873077393e-10
1.3969838619232178e-09
2002
-4.656612873077393e-10
-9.313225746154785e-10
2003
4.656612873077393e-10
9.313225746154785e-10
2004
-4.656612873077393e-10
-9.313225746154785e-10
2005
0.0
0.0
2006
-9.313225746154785e-10
0.0
2007
-9.313225746154785e-10
-9.313225746154785e-10
2008
-1.862645149230957e-09
-9.313225746154785e-10
2009
9.313225746154785e-10
1.862645149230957e-09
2010
0.0
0.0


In [37]:
# 10) Verificar igualdade entre demanda e oferta (niveis 12, 20, 51 e 68)
for t in range(2010,2021):
  print(t)
  [print(ios.system(str(t),x,'t').OA - ios.system(str(t),x,'t').DA) for x in ['12','20','51','68']]

2010
0.0
0.0
0.0
0.0
2011
0.0
-9.313225746154785e-10
0.0
9.313225746154785e-10
2012
1.862645149230957e-09
0.0
9.313225746154785e-10
0.0
2013
0.0
0.0
0.0
0.0
2014
0.0
0.0
0.0
0.0
2015
0.0
0.0
0.0
0.0
2016
0.0
0.0
0.0
0.0
2017
0.0
0.0
0.0
0.0
2018
0.0
-1.862645149230957e-09
0.0
0.0
2019
0.0
0.0
0.0
0.0
2020
-1.862645149230957e-09
0.0
1.862645149230957e-09
0.0


In [39]:
# 11) matrizes do sistema IO com setor household (coluna) e produto trabalho (linha)

#modificar a matriz de demanda intermediária
##inserir o bem trabalho na última linha damatriz de demanda intermediária
##bens estão nas linhas e setores nas colunas. Cada setor produz um unico bem
import numpy as np
from iotbr import io_system as ios
sys = ios.system('2010','68','t')
mY = sys.mY
vVA_table_rem = tru.read_var('2010','68','VA_table','t')['Remunerações'].values.reshape(-1, 1).T#renda das famílias (=Renda do Trabalho)
mZBarr =  np.concatenate((sys.mD_int_pb_qua, vVA_table_rem), axis=0)

##inserir setor trabalho na ultima coluna da matriz de demanda intermediária
vD_final_pb_qua_C_f = mY['C_f'].values #consumo das famílias a pb
vD_final_pb_qua_C_f_ = np.append(vD_final_pb_qua_C_f, 0).reshape(-1, 1)
mZBarr = np.concatenate((mZBarr, vD_final_pb_qua_C_f_), axis=1)


#reestimar a matris auxiliar A
vVBP = tru.read_var('2010','68','VA_table','t')['Valor da produção'].values
#devo somar 'vVA_table_rem' ou 'vVBP'????
vVBP = np.append(vVBP, np.sum(vVA_table_rem)).reshape(-1, 1).T
mABarr= np.zeros([68+1,68+1], dtype=float)
mABarr[:,:] = mZBarr[:,:]  / vVBP[0,:]

#reestimar a matriz de Leontief
mIBarr = np.eye(68 +1)
mLeontiefBarr = np.linalg.inv(mIBarr - mABarr)

print(mZBarr.shape)
mLeontiefBarr

(69, 69)


array([[1.05580354, 0.09072457, 0.0304036 , ..., 0.05456855, 0.108639  ,
        0.108639  ],
       [0.01734766, 1.06362242, 0.01701162, ..., 0.02880225, 0.05849285,
        0.05849285],
       [0.00873348, 0.01714415, 1.0832968 , ..., 0.00725779, 0.01406432,
        0.01406432],
       ...,
       [0.01606992, 0.02025271, 0.01161354, ..., 1.03154091, 0.06433127,
        0.06433127],
       [0.00948571, 0.01220092, 0.00667081, ..., 0.01808306, 1.0418943 ,
        0.0418943 ],
       [0.38056413, 0.48949792, 0.26763131, ..., 0.72548784, 1.68078882,
        1.68078882]])